# CADP tools queries 

## 2. SVL queries

In [79]:
import pandas as pd
import numpy as np 

In [80]:
file_name = "9M-5-[1-2001]-[a]-lcm.bcg"
file_path = f"../graph/bcg/{file_name}"

In [81]:
before_promo_period = "2018-09-01"
promo_period ="2018-12-01"
after_promo_period = ":2019-02-01"

In [82]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.classes_ = np.load(f'../../plots/encoders/{file_name.replace("bcg","out")}.npy')
promo_products = pd.read_csv("../../datasets/Total/promo_products.csv")
promo_products = f':{":".join(promo_products.ARTICLE_ID.astype(str).unique())}:'

In [83]:
promo_products

':3425901014890:3425901019178:3425900000962:3011449262970:3425901029788:3425901015859:3425901015835:3425901014944:3425901018188:3425901018201:3425901027432:3425901027449:3011449300559:3425901018140:3267025011375:3267025017650:3267025010606:3267025001406:3267021013236:3267021013229:3267025000836:3267021030738:3267025007262:3267021030837:3267025007293:3267021030530:3267021020531:3267025007361:3267025007460:3267021020432:3267025013072:3267025013065:3267025000072:3011449257037:3068320110783:3425901019260:3425901037172:3425901025872:3425901048086:3425901017969:3425901028170:3425901018249:3425901090993:3011449260457:3425901028187:3425901026251:3425901019505:3425901018232:3425909001007:3425909002011:3425909002943:3425900000436:3425901018720:3425901027487:3425901018195:3011449014005:3425901029320:3425901019734:3425901027869:3425901017976:3425901029290:'

### 2.1 Adoption query

In [84]:
query_name = "product_adoption.svl"
query_path = f"../queries/svl/{query_name}"
file_path = f'../graph/bcg/9M-5-[1-2001]-[a]-lcm'

In [87]:
%%bash -s $file_path 
export PATH=$PATH:$CADP/bin.x64:$CADP/com
svl product_adoption.svl $1 ":2018-09-01" ":2018-12-01" ":2019-02-01"


property PRODUCT_ADOPTION
 | Product adoption

TRUE

[Customers] [Promotional products] Period

[101392, 103904, 105649, 105769, 111061, 116517, 117912, 123642, 123949, 125615, 128265, 130464, 131834, 132260, 137060, 138325, 145566, 146201, 147772, 149886, 152266, 153534, 153850, 155710, 157047, 160513, 165324, 171799, 172457, 173559, 178285, 181027, 181964, 184070, 184841, 188482, 189419, 189637, 191747, 195200, 197939, 202124, 203487, 207309, 211606, 213211, 214303, 215374, 220829, 225819, 226924, 228704, 236344, 238881, 241262, 241363, 245072, 249602, 252901, 257663, 262086, 263432, 267956, 269558, 270927, 271256, 274467, 274953, 281256, 285626, 287447, 288714, 290288, 291564, 294576, 295124, 296219, 297113, 298742, 302709, 303696, 304157, 307874, 310380, 313472, 317586, 318322, 319166, 321180, 326483, 326781, 329769, 339428, 341341, 343312, 343449, 344970, 349622, 354817, 355592, 363045, 365174, 366567, 367316, 368517, 372225, 372638, 376752, 380944, 381797, 387373, 388999, 400788

###  2.2 Loyalty query 

In [88]:
%%bash -s $file_path
export PATH=$PATH:$CADP/bin.x64:$CADP/com
$CADP/com/svl product_loyalty.svl $1 ":3461025:7461024:" ":2018-12-01" ":2019-02-01"


property product_loyalty

TRUE

[Customers] [Promotional products] Period

[1044697] [3461025] :2018-12-01
[211249] [7461024] :2018-12-01
[575459, 1044697, 1163180] [3461025] :2018-12-01
[575459] [3461025] :2018-12-01



### 2.3 Pomotion only query

In [108]:
%%bash -s $file_path
export PATH=$PATH:$CADP/bin.x64:$CADP/com
$CADP/com/svl promotion_only.svl $1 ":2018-09-01" ":2018-12-01" ":2019-02-01"


property promotion_only

TRUE

[Customers] [Promotional products] Period




In [109]:
!gedit promotion_only.svl


## Draft

In [6]:
text = """
    Hello, i'm trying here ${a} to write ${b}
"""
with open('temp.txt',"w") as file:
    file.write(text)
!cat temp.txt


    Hello, i'm trying here ${a} to write ${b}


In [8]:
%%bash
a=12 b=14 envsubst< temp.txt > query.txt

In [1]:
!T0=:2018-09-01 T1=:2018-12-01 envsubst < ../queries/mcl/product_adoption.mcl > query.mcl

In [5]:
a = !bcg_open ../graph/bcg/9M-5-\[1-2001\]-\[a\]-lcm evaluator4 query.mcl 

In [11]:
for i in a[2:]:
#     C0,P0,C1,P1,C2,P2 = i.split()
    print(i.split())

[':100169,:100882,:102233,:103922,:105498,:105649,:106845,:107493,:107936,:108122,:108520,:108765,:109551,:111432,:112013,:112161,:112979,:113297,:113425,:113797,:114423,:114771,:115022,:115334,:115588,:116610,:117321,:117461,:117978,:119521,:120248,:120768,:120921,:121408,:122824,:123708,:123777,:123892,:124069,:124681,:125579,:126718,:128150,:128933,:129040,:129151,:130015,:130184,:130260,:131320,:131514,:131516,:132253,:132262,:132640,:132873,:133162,:133892,:133917,:135348,:136649,:136688,:137677,:137806,:138543,:138874,:139035,:140439,:140965,:141778,:142418,:143054,:144465,:145467,:146720,:148360,:149020,:149838,:150865,:151260,:151353,:152715,:152797,:153490,:153850,:154052,:154349,:154737,:154953,:156366,:156490,:157310,:157319,:157357,:157573,:158293,:158554,:158671,:159506,:160728,:160892,:162048,:163332,:164525,:165785,:167012,:167209,:167298,:167389,:167870,:168983,:170158,:170884,:171151,:171711,:171799,:172838,:172974,:173319,:173656,:174547,:176162,:176750,:177158,:17793